# Load the dataset

In [14]:
# load the correct dataset
import pandas as pd
data_100 = pd.read_pickle ('datasub_test_result_Morgan2Feat_Tanimoto.pkl')

In [15]:
data_100.head(1)

,Reaction SMILES,reactionID,react_smiles,rxn_smiles,template,Template Verification (True/False),canonical_rxn_smiles,dataset,Reaction Partner (s),Rank,Reaction,Similarity (proposed product vs recorded product),Similarity (recorded reactant vs recorded product)
38,[CH2:2]([c:3]1[cH:4][cH:5][cH:6][cH:7][cH:8]1)...,38,O=C1CN(c2ccc(Oc3ccccc3)cc2OCc2ccccc2)S(=O)(=O)N1,c1ccc(C[O:9][c:10]2[c:11]([N:23]3[CH2:24][C:25...,[c:2]-[O;H0;D2;+0:1]-[C;H2;D2;+0]-[c;H0;D3;+0]...,True,O=C1CN(c2ccc(Oc3ccccc3)cc2OCc2ccccc2)S(=O)(=O)...,test,[[K]],1,O=C1CN(c2ccc(Oc3ccccc3)cc2OCc2ccccc2)S(=O)(=O)...,1.0,0.622222


In [16]:
data_100.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43525 entries, 38 to 447750
Data columns (total 13 columns):
Reaction SMILES                                       43525 non-null object
reactionID                                            43525 non-null int64
react_smiles                                          43525 non-null object
rxn_smiles                                            43525 non-null object
template                                              43525 non-null object
Template Verification (True/False)                    43525 non-null bool
canonical_rxn_smiles                                  43525 non-null object
dataset                                               43525 non-null object
Reaction Partner (s)                                  43525 non-null object
Rank                                                  43525 non-null int64
Reaction                                              43525 non-null object
Similarity (proposed product vs recorded product)     435

# Set up fast filter

In [17]:
from api_client import APIClient

#place you askcos host name below
hostname = ''
client = APIClient(hostname, verify=False)  # verify=True also works for askcos-demo since we have a valid certificate

#input username and password below
client.client.auth = ('', '')

In [18]:
#convert to list
reaction_smiles_list = data_100 ['Reaction'].tolist()

#drop 9999 values
valueToBeRemoved1 = 9999
result1 = filter(lambda val: val !=  valueToBeRemoved1, reaction_smiles_list) 

#drop 9998 values
valueToBeRemoved2 = 9998
result = filter(lambda val: val !=  valueToBeRemoved2, result1) 

resultList = list (result)

#get it into list format
datalist = []
for item in resultList:
    inputDict = {}
    all_reactants, all_products = item.split('>>')
    inputDict ['reactants'] = all_reactants
    inputDict ['products'] = all_products
    datalist.append (inputDict)

endpoint = 'fast-filter'
requests = [(endpoint, data) for data in datalist]
result_list = [client.post(endpoint, data=data) for endpoint, data in requests]
tids = [result['task_id'] for result in result_list]
task_results = [client.get_result(tid, timeout=600, interval=5) for tid in tids]
outputList = []
for task_result in task_results:
    outputList.append(task_result['output'])

res = {resultList[i]: outputList[i] for i in range(len(resultList))}

fast_filter = []

for row in data_100.itertuples():
    if row [11] == 9999:
        fast_filter.append (9999)
    elif row [11] == 9998:
        fast_filter.append (9998)
    else:
        fast_filter.append (res[row[11]])

data_100['Fast Filter'] = fast_filter

In [19]:
data_100.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43525 entries, 38 to 447750
Data columns (total 14 columns):
Reaction SMILES                                       43525 non-null object
reactionID                                            43525 non-null int64
react_smiles                                          43525 non-null object
rxn_smiles                                            43525 non-null object
template                                              43525 non-null object
Template Verification (True/False)                    43525 non-null bool
canonical_rxn_smiles                                  43525 non-null object
dataset                                               43525 non-null object
Reaction Partner (s)                                  43525 non-null object
Rank                                                  43525 non-null int64
Reaction                                              43525 non-null object
Similarity (proposed product vs recorded product)     435

In [20]:
data_100.head()

,Reaction SMILES,reactionID,react_smiles,rxn_smiles,template,Template Verification (True/False),canonical_rxn_smiles,dataset,Reaction Partner (s),Rank,Reaction,Similarity (proposed product vs recorded product),Similarity (recorded reactant vs recorded product),Fast Filter
38,[CH2:2]([c:3]1[cH:4][cH:5][cH:6][cH:7][cH:8]1)...,38,O=C1CN(c2ccc(Oc3ccccc3)cc2OCc2ccccc2)S(=O)(=O)N1,c1ccc(C[O:9][c:10]2[c:11]([N:23]3[CH2:24][C:25...,[c:2]-[O;H0;D2;+0:1]-[C;H2;D2;+0]-[c;H0;D3;+0]...,True,O=C1CN(c2ccc(Oc3ccccc3)cc2OCc2ccccc2)S(=O)(=O)...,test,[[K]],1,O=C1CN(c2ccc(Oc3ccccc3)cc2OCc2ccccc2)S(=O)(=O)...,1.000000,0.622222,0.995744
41,[CH3:1][c:2]1[n:3][cH:4][cH:5][c:6](-[c:8]2[cH...,41,Cc1cc(-c2ccc(CC(=O)O)cc2)ccn1,O=[C:15]([CH2:14][c:11]1[cH:10][cH:9][c:8](-[c...,[C:1]-[C;H0;D3;+0:2](=[O;H0;D1;+0])-[OH;D1;+0:...,True,Cc1cc(-c2ccc(CC(=O)O)cc2)ccn1>>Cc1cc(-c2ccc(CC...,test,[Nc1ccc(N2CCOCC2)nc1],16,Cc1cc(-c2ccc(CC(=O)O)cc2)ccn1.COC(=O)c1ccc(N)c...,0.636364,0.430108,0.999997
44,[Br:1][c:2]1[n:3][c:4]([CH:8]([CH2:9][F:10])[B...,44,FCC(Br)c1cccc(Br)n1,Br[CH:8]([c:4]1[n:3][c:2]([Br:1])[cH:7][cH:6][...,[#7;a:1]:[c:2]-[CH;D3;+0:3](-[Br;H0;D1;+0])-[C...,True,FCC(Br)c1cccc(Br)n1>>FCC(c1cccc(Br)n1)N1CCOCC1,test,[C1COCCN1],9999,9999,0.392857,0.461538,9999.000000
47,[CH3:27][CH2:28][N:29]([CH2:30][CH3:31])[CH2:3...,47,O=C(O)c1c[nH]c2ccccc12,O[C:2](=[O:3])[c:4]1[cH:5][nH:6][c:7]2[cH:8][c...,[#7;a:5]:[c:4]:[c:3]-[C;H0;D3;+0:1](=[O;D1;H0:...,True,O=C(O)c1c[nH]c2ccccc12>>CCOC(=O)Cc1ccc(NC(=O)c...,test,[CCOC(=O)Cc1ccc(N)c(Cl)c1],3,O=C(O)c1c[nH]c2ccccc12.CC(C)(C)c1cccc(N)c1>>CC...,0.662500,0.312500,0.999809
55,[Al+3:2].[ClH:20].[H-:1].[H-:4].[H-:5].[H-:6]....,55,CC(C)C(C(=O)O)c1ccccc1,O[C:14]([CH:13]([c:7]1[cH:8][cH:9][cH:10][cH:1...,[C:1]-[C;H0;D3;+0:2](=[O;H0;D1;+0:3])-[O;H1;D1...,True,CC(C)C(C(=O)O)c1ccccc1>>CC(C)C(CO)c1ccccc1,test,[],9,CC(C)C(C(=O)O)c1ccccc1>>CC(C)C(CO)c1ccccc1,1.000000,0.545455,0.923400


# Fast-filter function check

In [21]:
#randomly sample 10 entries
data_100_sample = data_100.sample (10)

reactions = data_100_sample['Reaction'].tolist()
ff_score = data_100_sample['Fast Filter'].tolist()

result_zip = zip (reactions, ff_score)

for item in result_zip:
    print ('The reaction is: {}'.format (item[0]))
    print ('The fast filter score is: {}'.format (item[1]))
    print ('-'*50)

The reaction is: Cc1ccccc1-n1c(CCl)nc2cccc(Cl)c2c1=O.Sc1ncnc2nc[nH]c12>>Cc1ccccc1-n1c(CSc2ncnc3[nH]cnc23)nc2cccc(Cl)c2c1=O
The fast filter score is: 0.999983191
--------------------------------------------------
The reaction is: O=C(NC(Cc1ccc(F)cc1)C(=O)O)c1cc2cc(Cl)cnc2[nH]1.OC1CCNC1>>O=C(NC(Cc1ccc(F)cc1)C(=O)N1CCC(O)C1)c1cc2cc(Cl)cnc2[nH]1
The fast filter score is: 0.999986529
--------------------------------------------------
The reaction is: COc1cc(C(=O)N(C)c2ccc(C)cc2OCCCCN)ccc1NC(=O)c1cccc2[nH]c(C)nc12.CC(=O)OC(C)=O>>COc1cc(C(=O)N(C)c2ccc(C)cc2OCCCCNC(C)=O)ccc1NC(=O)c1cccc2[nH]c(C)nc12
The fast filter score is: 0.99991715
--------------------------------------------------
The reaction is: CCOC(=O)CC(c1cc(F)ccc1F)S(=O)(=O)c1ccc(Cl)cc1>>O=S(=O)(c1ccc(Cl)cc1)C(CCO)c1cc(F)ccc1F
The fast filter score is: 0.993128538
--------------------------------------------------
The reaction is: COc1ccc2c(Nc3c(Cl)cncc3Cl)cc(=O)[nH]c2c1OCCCCCCCl.C1CCNC1>>COc1ccc2c(Nc3c(Cl)cncc3Cl)cc(=O)[nH]c2c1OCCC

# Save the fast filter score

In [22]:
data_100.to_pickle ('datasub_test_result_Morgan2Feat_Tanimoto_FastFilter.pkl')